## Oracle 23ai Vector Store와 Solar Mini Chat LLM을 활용한 RAG 구현

### 1. Oracle Database 23ai 접속

In [65]:
import os
import time
import oracledb
import configparser
config = configparser.ConfigParser()
config.read("oci.env")

username = config["DATABASE"]["USERNAME"]
password = config["DATABASE"]["PASSWORD"]
host = config["DATABASE"]["HOST"]
port = config["DATABASE"]["PORT"]
service_name = config["DATABASE"]["SERVICE_NAME"]
table_name = config["DATABASE"]["TABLE_NAME_CV_LANG"]
compartment_id = config["OCI"]["compartment_id"]
dsn=host+":"+port+"/"+service_name
upstage_api_key=config["APIKEY"]["UPSTAGE_API_KEY"]

try:
    oracledb.init_oracle_client()
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("\nConnected to the Oracle Database 23.4.\n")
except Exception as e:
    print(e)
    print("\nConnection failed!\n")
    


Connected to the Oracle Database 23.4.



### 2. Load the document
#### 텍스트 추출: UpstageLayoutAnalysisLoader, RecursiveCharacterTextSplitter

In [66]:
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain_text_splitters import (Language, RecursiveCharacterTextSplitter)

os.environ["UPSTAGE_API_KEY"] = upstage_api_key

pdf_file="/home/opc/23ai_rag_demo/app/pdfs/SPRi_AI_Brief_4.pdf"

chunks_with_mdata = []

layzer = UpstageLayoutAnalysisLoader(pdf_file, split="page")
docs = layzer.load()  # or layzer.lazy_load()
print(f"Number of pages loaded: {len(docs)}")

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=500, chunk_overlap=100, language=Language.HTML
)

chunks = text_splitter.split_documents(docs)

print(f"Doc {id}: chunks# {len(chunks)}")

for ic, chunk in enumerate(chunks, start=1):
    counter += 1  
    chunk_metadata = doc.metadata.copy()  
    chunk_metadata['id'] = str(counter)  
    chunk_metadata['document_id'] = str(document_num)
    # chunk_metadata['document_summary'] = str(summ[0])
    chunks_with_mdata.append(Document(page_content=str(chunk), metadata=chunk_metadata))

print(f"Doc {id}: page_content: { chunks_with_mdata[4].page_content} metadata: {chunks_with_mdata[4].metadata}")

Number of pages loaded: 21
Doc 1: chunks# 115
Doc 1: page_content: page_content='<p id='5' data-category='paragraph' style='font-size:18px'>2. 기업/산업</p> <br><p id='6' data-category='list' style='font-size:14px'>▹ 스태빌리티AI, 차세대 이미지 생성 AI ‘스테이블 디퓨전 3’ 프리뷰 버전 공개 ················· 7<br>▹ 오픈AI, 일론 머스크의 소송과 정부 조사, 저작권 소송으로 법적 부담 증가 ···················· 8<br>▹ 휴머노이드 로봇 스타트업 피규어, 오픈AI의 기술 접목한 ‘피규어01’ 로봇 시연 ············· 9<br>▹ 일론 머스크의 xAI, AI 챗봇 ‘그록’을 오픈소스로 공개 ···················································· 10</p> <br>' metadata={'page': 2} metadata: {'SOURCE MIME TYPE': 'application/pdf', 'creation date': '4/8/2024 12:46:55 AM', 'author': 'spri', 'revision date': '4/8/2024 12:46:55 AM', 'Creator': '\rHwp 2018 10.0.0.13764', 'publisher': 'Hancom PDF 1.3.0.542', 'PDFVersion': '\r1.4', '_oid': '6692b6706aa918ee066f7ec26e173d99', '_file': '/home/opc/23ai_rag_demo/app/pdfs/SPRi_AI_Brief_4.pdf', 'id': '324', 'document_id': '1'}


### 3. Embedding 및 벡터 데이터베이스에 입력
#### Embedding Model: solar-embedding-1-large

In [67]:
from langchain_upstage import UpstageEmbeddings

s1time = time.time()
vector_store = OracleVS.from_documents(chunks_with_mdata, UpstageEmbeddings(model="solar-embedding-1-large"), client=connection, table_name=table_name_with_strategy, distance_strategy=distance_strategy)

### Create Oracle HNSW Index
oraclevs.create_index(client=connection,vector_store=vector_store, params={
    "idx_name": "hnsw"+table_name_with_strategy, "idx_type": "HNSW"
})
s2time = time.time()

if vector_store is not None:
    print( f"Documents loading, chunking and generating embeddings are complete.\nVectorizing and inserting chunks duration: {round(s2time - s1time, 1)} sec.")
else:
    print("\nFailed to get the VectorStore populated.\n")

Documents loading, chunking and generating embeddings are complete.
Vectorizing and inserting chunks duration: 22.4 sec.


### 4. Retriever 생성 및 및 유사도 검색
#### Embedding Model: OCI GenAI cohere.embed-multilingual-v3.0

In [68]:
vector_store = OracleVS(client=connection, 
                        embedding_function=UpstageEmbeddings(model="solar-embedding-1-large"), 
                        table_name=table_name_with_strategy, 
                        distance_strategy=distance_strategy)

user_question = ("최초의 AI법은 언제 통과 되었니? 출처나 참고 문서도 같이 알려줘.");

if user_question:
    s1time =  time.time()
    result_chunks = vector_store.similarity_search(user_question)
    s2time = time.time()
    print(f"result_chunks={result_chunks}\nSearch for the user question in the Oracle Database 23ai and return similar chunks duration: {round(s2time - s1time, 1)} sec.")


result_chunks=[Document(metadata={'SOURCE MIME TYPE': 'application/pdf', 'creation date': '4/8/2024 12:46:55 AM', 'author': 'spri', 'revision date': '4/8/2024 12:46:55 AM', 'Creator': '\rHwp 2018 10.0.0.13764', 'publisher': 'Hancom PDF 1.3.0.542', 'PDFVersion': '\r1.4', '_oid': '6692b6706aa918ee066f7ec26e173d99', '_file': '/home/opc/23ai_rag_demo/app/pdfs/SPRi_AI_Brief_4.pdf', 'id': '322', 'document_id': '1'}, page_content="page_content='<p id='4' data-category='list' style='font-size:14px'>▹ 유럽의회 본회의에서 세계 최초의 AI 법 통과 ····································································· 1<br>▹ 유럽평의회, AI·인권·민주주의·법치에 관한 기본 협약에 합의 ············································· 2<br>▹ 프랑스 AI 위원회, 정부에 5년간 연 50억 유로의 AI 투자 권고 ······································· 3<br>▹ 뉴욕주를 비롯한 미국 주정부들, AI 선거조작 방지법안 추진 ············································· 4' metadata={'page': 2}"), Document(metadata={'SOURCE MIME TYPE': 'application/pdf', 'creation date': '4/8/2024 12:46:55 AM', 'author': 'spri', '

### 5. Langchain RAG
#### Vector Store Retriver, solar-1-mini-chat LLM 모델 사용

In [75]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from openai import OpenAI

chat = ChatUpstage(model="solar-1-mini-chat")

message = [
    (
        "system",
        """
        질문-답변 업무를 돕는 AI 어시스턴트입니다. 
        문서의 내용을 참고해서 답변해 주세요.:
        \n\n
        {context}",
        """
    ),
    ("human", "{human}"),
]

prompt = ChatPromptTemplate.from_messages(message)
        
chain = {
    "context": vector_store.as_retriever(),
    "human": RunnablePassthrough(),
} | prompt | chat | StrOutputParser()

s1time=time.time()
response = chain.invoke(user_question)
s2time=time.time()

# Groundedness Check
user_content = '\n\n'.join([d.page_content for d in vector_store.similarity_search(user_question)])

upstage_client = OpenAI(
    api_key=upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

s3time=time.time()
groundedness_check_response = upstage_client.chat.completions.create(
    model="solar-1-mini-groundedness-check",
    messages=[
        {
          "role": "user",
          "content": user_content
        },
        {
          "role": "assistant",
          "content": response
        }
    ]
)
s4time=time.time()

print( f"response={response}\nSend user question and ranked chunks to LLM and get answer duration: {round(s2time - s1time, 1)} sec.\nGroundedness check result={groundedness_check_response.choices[0].message.content}\nGroundedness check duration: {round(s4time - s3time, 1)} sec.")



response=최초의 AI법인 AI Act는 2024년 3월 13일에 유럽의회 본회의에서 통과되었습니다. 출처는 European Parliament의 "Artificial Intelligence Act: MEPs adopt landmark law"이며, 2024년 3월 13일에 발표되었습니다. 또한, European Commission의 "European AI Office"라는 문서에서도 2024년 2월 21일에 발표되어 관련 정보를 확인할 수 있습니다.
Send user question and ranked chunks to LLM and get answer duration: 2.2 sec.
Groundedness check result=grounded
Groundedness check duration: 0.5 sec.
